In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [9]:
os.chdir(r'C:\Users\longj\Desktop\analyst_tone')

In [10]:
project_dir=Path.cwd()
project_dir

WindowsPath('C:/Users/longj/Desktop/analyst_tone')

In [8]:

import os
import copy
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertConfig
from bertModel import BertClassification, dense_opt
from datasets import text_dataset, financialPhraseBankDataset
import argparse
from sklearn.metrics import f1_score

In [26]:
labels = {0:'neutral', 1:'positive',2:'negative'}
num_labels= len(labels)
vocab = "finance-uncased"
#vocab_path = "/home/root/new_bert_vocab_uncased_8000"
vocab_path = project_dir/'vocab'
pretrained_weights_path = project_dir/'pretrained_weights' # this is pre-trained FinBERT weights
fine_tuned_weight_path = project_dir/'fine_tuned.pth'      # this is fine-tuned FinBERT weights
max_seq_length=256
device='cuda:0'

In [20]:
model = BertClassification(weight_path= pretrained_weights_path, num_labels=num_labels, vocab=vocab)

In [24]:
model.load_state_dict(torch.load(fine_tuned_weight_path,map_location='cuda:0'))


<All keys matched successfully>

In [27]:
model.to(device)

BertClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )


In [77]:
sentences = ['To the extent that FASL is unable to secure the necessary funds forFASL II, the Company may be required to contribute cash or guar-antee third-party loans . . . In the event the Company is unable toobtain the external financing necessary to meet its covenants underthe Credit Agreement, it will also be unable to fund its capital in-vestments planned . . . There can be no assurance that the Companywill be able to obtain the funds necessary to fulfill these obligationsand any such failure would have a material adverse effect on theCompany.']

In [78]:
tokenizer = BertTokenizer(vocab_file = vocab_path, do_lower_case = True, do_basic_tokenize = True)
for sent in sentences: 
    tokenized_sent = tokenizer.tokenize(sent)
    if len(tokenized_sent) > max_seq_length:
        tokenized_sent = tokenized_sent[:max_seq_length]
    
    ids_review  = tokenizer.convert_tokens_to_ids(tokenized_sent)
    mask_input = [1]*len(ids_review)        
    padding = [0] * (max_seq_length - len(ids_review))
    ids_review += padding
    mask_input += padding
    input_type = [0]*max_seq_length
    
    input_ids = torch.tensor(ids_review).to(device).reshape(-1, 256)
    attention_mask =  torch.tensor(mask_input).to(device).reshape(-1, 256)
    token_type_ids = torch.tensor(input_type).to(device).reshape(-1, 256)
    
    with torch.set_grad_enabled(False):
        outputs = model(input_ids, token_type_ids, attention_mask)
        outputs = F.softmax(outputs,dim=1)
        print(sent, '\nBERT predicted sentiment: ', labels[torch.argmax(outputs).item()], '\n')

To the extent that FASL is unable to secure the necessary funds forFASL II, the Company may be required to contribute cash or guar-antee third-party loans . . . In the event the Company is unable toobtain the external financing necessary to meet its covenants underthe Credit Agreement, it will also be unable to fund its capital in-vestments planned . . . There can be no assurance that the Companywill be able to obtain the funds necessary to fulfill these obligationsand any such failure would have a material adverse effect on theCompany. 
BERT predicted sentiment:  negative 

